In [215]:
import pandas as pd 
import numpy as np
#tdqm = progress bar
from tqdm import tqdm
from datetime import datetime

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn import utils

#number of rows
chunksize = 10000 

path = 'C:/Users/sirus/Downloads/train.csv'

#columns to read from the data
#Todo :split date into meaningful data
cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','fare_amount']

#list to hold the batch dataframe
df_list = []

for df_chunk in tqdm(pd.read_csv(path,usecols=cols, chunksize=chunksize,nrows=1000)):
    df_list.append(df_chunk) 


# Merge all dataframes into one dataframe
data = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list, df_chunk

#replace missing fields with the mean of that column
data.fillna(data.mean(), inplace=True)

feature_cols = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']

X = data[feature_cols] 
y = data['fare_amount']

X = preprocessing.scale(X)
y = preprocessing.scale(y)

# Delete the dataframe list to release memory
del data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                 random_state=2)
#55millions rows it = iteration(nrows per it) x nrows = total rows

print(X.shape)
print(y.shape)

#preprocessing.LabelEncoder()-convert string or float values to 0 .. n classes.
lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_encoded = lab_enc.fit_transform(y)

print(y[0:5])
print(y_encoded[0:5])

1it [00:00, 252.27it/s]


(1000, 5)
(1000,)
[-0.73406108  0.57188379 -0.60767931 -0.39704305 -0.64980657]
[ 8 63 13 23 11]


In [216]:
#linear regression can also use y_train
my_linreg = LinearRegression()
my_linreg.fit(X_train, y_train_encoded)
y_prediction = my_linreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

33.16885257680002


In [217]:
my_logreg = LogisticRegression(solver='liblinear', multi_class='auto')
my_logreg.fit(X_train, y_train_encoded)
y_prediction = my_logreg.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

9.251622315923028


In [218]:
my_decisiontree = DecisionTreeClassifier(random_state=5)
my_decisiontree.fit(X_train, y_train_encoded)
y_prediction = my_decisiontree.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_prediction)
rmse = np.sqrt(mse)
print(rmse)

41.47568557513889


Cross Validation

In [219]:
score_list = cross_val_score(my_linreg, X, y_encoded, cv=10)
score = score_list.mean()
print('KNN: '+str(score))

KNN: -24.066076941544654


In [220]:
score_list = cross_val_score(my_logreg, X, y_encoded, cv=10)
score = score_list.mean()
print('Logistic Regression: '+str(score))

C:\Users\sirus\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Logistic Regression: 0.0533717188501137


In [221]:
score_list = cross_val_score(my_decisiontree, X, y_encoded, cv=10)
score = score_list.mean()
print('Decision Tree: '+str(score))

C:\Users\sirus\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Decision Tree: 0.03920513600717432
